In [5]:
from pydub import AudioSegment
from pydub.silence import detect_nonsilent, detect_silence
from pydub.silence import split_on_silence
import numpy as np
import ffmpeg

from os import listdir
from os.path import isfile, join

#adjust target amplitude
def match_target_amplitude(sound, target_dBFS):
    change_in_dBFS = target_dBFS - sound.dBFS
    return sound.apply_gain(change_in_dBFS)

In [6]:
with open("./records/_records.txt") as f:
    contents = f.readlines()
only_files = [f for f in listdir("./records/") if isfile(join("./records/", f))]
only_files = list(filter(lambda x: ".wav" in x, only_files))

In [7]:
data = []
for file in only_files:
#Convert wav to audio_segment
    name = file[:file.rfind(".")]
    audio_segment = AudioSegment.from_wav("./records/" + name + ".wav")

    #normalize audio_segment to -20dBFS 
    normalized_sound = match_target_amplitude(audio_segment, -20.0)
    print("length of audio_segment = {} seconds".format(len(normalized_sound)/1000))

    #Print detected non-silent chunks, which in our case would be spoken words.
    nonsilent_data = detect_silence(normalized_sound, min_silence_len=500, silence_thresh=-25, seek_step=1)

    print("start, stop")
    flattened = []
    combined = []
    for tuple in nonsilent_data:
        for value in tuple:
            flattened.append(value)

    flattened = np.array(flattened)

    print(name)
    res = list(filter(lambda x: name in x, contents))[0]
    mood = 0
    if "bad" in res:
        mood = 0
    if "neutral" in res:
        mood = 1
    if "good" in res:
        mood = 2
    flattened_pmeta = [mood, flattened]
    flattened_pmeta = np.array(flattened_pmeta)
    data.append(flattened_pmeta)

data = np.array(data)

length of audio_segment = 15.0 seconds
start, stop
2022-12-16_16-58-43
length of audio_segment = 15.0 seconds


C:\Users\ioci\AppData\Local\Temp\ipykernel_11448\2102238370.py:33: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  flattened_pmeta = np.array(flattened_pmeta)


start, stop
2022-12-16_16-58-58
length of audio_segment = 15.0 seconds
start, stop
2022-12-16_16-59-13
length of audio_segment = 15.0 seconds
start, stop
2022-12-16_16-59-28
length of audio_segment = 15.0 seconds
start, stop
2022-12-16_16-59-43
length of audio_segment = 15.0 seconds
start, stop
2022-12-16_16-59-58
length of audio_segment = 15.0 seconds
start, stop
2022-12-16_17-00-13
length of audio_segment = 15.0 seconds
start, stop
2022-12-16_17-00-28
length of audio_segment = 15.0 seconds
start, stop
2022-12-16_17-00-43
length of audio_segment = 15.0 seconds
start, stop
2022-12-16_17-00-58
length of audio_segment = 15.0 seconds
start, stop
2022-12-16_17-01-13
length of audio_segment = 15.0 seconds
start, stop
2022-12-16_17-01-28
length of audio_segment = 15.0 seconds
start, stop
2022-12-16_17-05-50
length of audio_segment = 15.0 seconds
start, stop
2022-12-16_17-06-05
length of audio_segment = 15.0 seconds
start, stop
2022-12-16_17-06-20
length of audio_segment = 15.0 seconds
start,

In [8]:
import pandas as pd

# Convert `X` to data frame
df = pd.DataFrame(data)

# Rename columns
df.columns = ['feature_' + str(i + 1) for i in range(df.shape[1])]

# Convert the feature with lists inside to long format
features = df['feature_2'].explode().to_frame()

# Create counter by observation so we can pivot
features['observation_id'] = features.groupby(level=0).cumcount()

# Convert to dataset and rename all columns
features = features.pivot(columns='observation_id', values='feature_2').fillna(0)
features = features.add_prefix('list_element_')

# Drop `feature_5` from X
df.drop(columns='feature_2', axis=1, inplace=True)

# Concatenate X and x together
df = pd.concat([df, features], axis=1)

In [9]:
df.head(5)

,feature_1,list_element_0,list_element_1,list_element_2,list_element_3,list_element_4,list_element_5,list_element_6,list_element_7,list_element_8,list_element_9,list_element_10,list_element_11,list_element_12,list_element_13,list_element_14,list_element_15
0,2,0,5756,8600,9323,9835,15000,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X = df.drop(["feature_1"],axis=1)
y = df["feature_1"]
y=y.astype('int')

In [12]:
# Carry on as before
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.35, random_state = 42)

In [13]:
from sklearn.svm import SVC
model  = SVC()
model.fit(X_train, y_train)
predicted = model.predict(X_test)
score = model.score(X_test, y_test)
score

0.5192307692307693

In [14]:
from sklearn.ensemble import GradientBoostingClassifier
model  = GradientBoostingClassifier()
model.fit(X_train, y_train)
predicted = model.predict(X_test)
score = model.score(X_test, y_test)
score

0.46153846153846156

In [15]:
SVC_scores = []
GBC_scores = []

from statistics import mean

for i in range(1, 300):
    X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.33, random_state = i)

    model  = SVC()
    model.fit(X_train, y_train)
    predicted = model.predict(X_test)
    SVC_scores.append(model.score(X_test, y_test))

    model  = GradientBoostingClassifier()
    model.fit(X_train, y_train)
    predicted = model.predict(X_test)
    GBC_scores.append(model.score(X_test, y_test))

print(f'SVC average: {mean(SVC_scores)}')
print(f'GBC average: {mean(GBC_scores)}')

SVC average: 0.5067230905740222
GBC average: 0.5194867244556686
